## Linear Programming with Datamodel

Consider the following example, 

Given system constraints:

                                          2x + 4y >= 230 
                                          3x + 2y <= 190 
                                          x >= 0 
                                          y >= 0,

Maximize objective function:
                            
                                          f(x) = 5x + 3y 


You need to find real numbers for x and y in such a way that it satisfies constraints and maximizes the objective function.

#### Environment Setup
First, let's check if we have a GPU available in system. This is only applicable if you are running server through this notebook. Else comment the next cell.


In [ ]:
# Check for GPU
!nvidia-smi




#### Install dependencies

In [2]:
# Install cuOpt

# Enable this in case you are running this in google colab or such places where cuOpt is not yet installed

# This would be incase underlying system is cuda-10.X 
# !pip install --user cuopt-server-cu11==25.5.* cuopt-sh-client==25.5.*

# This would be incase underlying system is cuda-12.x
# !pip install --user cuopt-server-cu12==25.5.* cuopt-sh-client==25.5.*

#### Run cuOpt server in background

Please disable the next cell in case you are trying to test against a server running on a different machine.

In [ ]:
import subprocess
import atexit
import signal
import os
import psutil

def is_cuopt_server_running():
    for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
        try:
            if 'python' in proc.info['name'] and \
               any('cuopt_server.cuopt_service' in cmd for cmd in proc.info['cmdline'] if cmd):
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            pass
    return False

# Only start server if it's not already running
if not is_cuopt_server_running():
    # Start cuOpt server in background
    server_process = subprocess.Popen(['python', '-m', 'cuopt_server.cuopt_service'],
                                    stdout=subprocess.PIPE,
                                    stderr=subprocess.PIPE)
    
    def cleanup_server():
        try:
            server_process.terminate()
            server_process.wait(timeout=5)  # Wait up to 5 seconds for graceful termination
            print("Terminated cuOpt server")
        except (subprocess.TimeoutExpired, ProcessLookupError):
            # If graceful termination fails, force kill
            try:
                server_process.kill()
                print("Killed cuOpt server")
            except ProcessLookupError:
                pass

    atexit.register(cleanup_server)
    print("Started cuOpt server")
else:
    print("cuOpt server is already running")


In [ ]:
import numpy as np
import cuopt_mps_parser

from data_model import DataModel
from cuopt_sh_client import ThinClientSolverSettings

problem_data = {}

dm = DataModel()
ss = ThinClientSolverSettings()

### Set Constraint Matrix

If the constraints are:
                    
                     2x + 4y >= 230 
                     3x + 2y <= 190 

Constraints are depicted in [CSR](https://docs.nvidia.com/nvpl/_static/sparse/storage_format/sparse_matrix.html#compressed-sparse-row-csr) format. The constraints can be transformed to the CSR matrix as follows:

In [5]:
offsets = np.array([0, 2, 4], dtype=np.int32)
indices = np.array([0, 1, 0, 1], dtype=np.int32)
coefficients = np.array([2.0, 4.0, 3.0, 2.0], dtype=np.float64)

dm.set_csr_constraint_matrix(coefficients, indices, offsets)

The offsets indicate the starting point of each constraint in the coefficients array and indices indicate the variables for each coefficient

### Set Constraint Bounds
If the constraints are as follows:
                    
                     2x + 4y >= 230     
                     3x + 2y <= 190 
                     
You need to define ``upper_bounds`` and ``lower_bounds`` of all the constraints, each value signifies the upper or lower bound of each constraint respective to its index. 

In [6]:
upper_bounds = np.array([np.inf, 190], dtype=np.float64)
lower_bounds = np.array([230, -np.inf], dtype=np.float64)

dm.set_constraint_lower_bounds(lower_bounds)
dm.set_constraint_upper_bounds(upper_bounds)

``inf`` - infinity and ``-inf`` - negative infinity are used when there is no explict upper or lower bound.

### Set Variable Bounds

Variables:

                      x >= 0 
                      y >= 0

Define the variable bounds similar to constraint bounds and optionally set the variable types

In [7]:
var_upper_bounds = np.array([np.inf, np.inf], dtype=np.float64)
var_lower_bounds = np.array([0, 0], dtype=np.float64)
dm.set_variable_lower_bounds(var_lower_bounds)
dm.set_variable_upper_bounds(var_upper_bounds)

### Set Objective Data

Objective:

                       f(x) = 5x + 3y
Pass coefficents for objective data and also set whether it needs to be maximized or minimized.

In [8]:
objective_coefficients = np.array([5, 3], dtype=np.float64)

dm.set_objective_coefficients(objective_coefficients)
dm.set_maximize(True)

### Set Variable Names

This is optional, but it helps users to navigate the result.



In [9]:
dm.set_variable_names(np.array(["x", "y"]))

### Set Variable Types
Set variable types, "I" - ``Integer`` "C" - ``Continuous``

The default is Continuous so this step is optional but shown here for illustration

In [10]:
dm.set_variable_types(np.array(["C", "C"]))

### Set Solver Configuration

The solver configuration can be fine-tuned for optimization and runtimes.

In [11]:

ss.set_parameter("time_limit", 1)
ss.set_optimality_tolerance(0.0001)

### Solve the Problem

cuOpt service endpoints can be triggered as shown in the [thin client example for self-hosted](../self-hosted-client/sh-cli-example.html#lp-example).

Use this data and invoke the cuOpt endpoint, which would return values for ``x`` and ``y``.

The following example shows how to use a locally hosted server:

In [12]:
data = cuopt_mps_parser.toDict(dm)
data["solver_config"] = ss.toDict()

# If we did not set variable types, toDict() will return
# an empty array for variable types. Remove it from the data.
if len(data["variable_types"]) == 0:
    del data["variable_types"]

In [ ]:
import json
from cuopt_sh_client import CuOptServiceSelfHostClient

# If cuOpt is not running on localhost:5000, edit ip and port parameters
# polling_timeout set to None will cause the client to poll automatically
# until there is a result or an error occurs. You may set polling_timeout
# to a numeric value in seconds (default is 600)
cuopt_service_client = CuOptServiceSelfHostClient(
    ip="localhost",
    port=5000,
    polling_timeout=None
)

solution = cuopt_service_client.get_LP_solve(data, response_type="dict")

print(json.dumps(solution, indent=4))

``Status - 1`` corresponds to ``Optimal solution is available``.

``` json
{
    "response": {
        "solver_response": {
            "status": 1,
            "solution": {
                "problem_category": 0,
                "primal_solution": [
                    37.5,
                    38.75000000000001
                ],
                "dual_solution": [
                    -0.12500016770509828,
                    -1.750000111803399
                ],
                "primal_objective": 303.75,
                "dual_objective": 6.3805226600279e-310,
                "solver_time": 0.019212961196899414,
                "vars": {
                    "x": 37.5,
                    "y": 38.75000000000001
                },
                "lp_statistics": {
                    "primal_residual": 6.37992675089215e-310,
                    "dual_residual": 6.38052266003797e-310,
                    "gap": 6.37992675097e-310,
                    "nb_iterations": 2
                },
                "reduced_cost": [
                    0.0,
                    0.0
                ],
                "milp_statistics": {}
            }
        },
        "total_solve_time": 0.08559036254882812
    },
    "reqId": "64ba4d53-1417-4f9f-96f6-299621147bce"
}
```

### License

SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved. 

SPDX-License-Identifier: MIT

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
